In [23]:
import torch

## Tensory

In [3]:
tensor0d = torch.tensor(1) # zerowymiarowy tensor - skalar
tensor1d = torch.tensor([1,2,3]) # jednowymiarowy tensor - wektor
tensor2d = torch.tensor([[1,2], [3,4]]) # dwówymiarowy tensor - macierz
tensor3d = torch.tensor([[1,2],[3,4],[5,6],[7,8]]) # trójwymiarowy tensor

In [4]:
tensor1d.dtype

torch.int64

In [5]:
print(tensor2d)

tensor([[1, 2],
        [3, 4]])


## Automatyczne różniczkowanie

### Obliczanie gradientów przez silnik autograd

In [2]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1*w1+b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a,y)
grad_L_w1 = grad(loss, w1, retain_graph = True) # zostawiamy graf w pamięci na potrzeby testu
grad_L_b = grad(loss, b, retain_graph = True)

In [4]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [5]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


## Wielowarstwowy perceptron z dwiema warstawami ukrytymi

In [11]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs): # zakodowanie liczby wejść i wyjść
        super().__init__()

        self.layers = torch.nn.Sequential(

            #pierwsza warstwa ukryta
            torch.nn.Linear(num_inputs, 30), #liczba węzłów wejściowych i wyjściowych
            torch.nn.ReLU(), #nieliniowa funkcja aktywacji RELU

            #druga warstwa ukryta
            torch.nn.Linear(30,20), #liczba węzłów wyjściowych odpowiada liczbie węzłów wejsciowych kolejnej warstwy

            #warstwa wyjściowa
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [4]:
model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [5]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Łączna liczba możliwych parametrów do wyuczenia w modelu: ", num_params)

Łączna liczba możliwych parametrów do wyuczenia w modelu:  2213


In [6]:
#Dostęp do macierzy parametrów
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.1164, -0.0047,  0.0048,  ..., -0.1265, -0.0545,  0.1074],
        [-0.0807,  0.0959, -0.0652,  ..., -0.0811,  0.1185,  0.0683],
        [ 0.1100,  0.0192,  0.0574,  ..., -0.0700,  0.0737, -0.0068],
        ...,
        [ 0.0903, -0.0795,  0.0141,  ...,  0.0297,  0.0196,  0.1410],
        [-0.0097,  0.0446,  0.1025,  ..., -0.0299, -0.0915,  0.0265],
        [-0.1334,  0.0431,  0.0590,  ..., -0.0035,  0.0518, -0.0742]],
       requires_grad=True)


In [ ]:
# Kiedy szkolenie nie ejst wymagane można je wyłączyć uzywając menadzera kontekstu
with torch.no_grad():
    out = model(X)

## Mechanizm ładujacy dane

### Utworznie prostego zbioru danych

In [2]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0,0,0,1,1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, 2.8]
])

y_test = torch.tensor([0,1])

### Niestandardowa klasa dataset

In [3]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X,y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]


In [4]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [5]:
print(len(train_ds))

5


### Komponent ładowania danych

In [8]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle=True, #przetasowanie danych
    num_workers=0, #liczba procesów w tle
    drop_last=True
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle=False, # tasowanie zbioru testowego nie jest konieczne
    num_workers=0
)

In [9]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Partia {idx+1}:", x,y)

Partia 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Partia 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


## Pętla szkoleniowa

In [13]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2) # zbiór danych ma 2 cechy i dwie klasy
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoka: {epoch+1:03d}/{num_epochs:03d}"
              f" | Partia {batch_idx:03d}/{len(train_loader):03d}"
              f" | Strata zbioru szkoleniowego: {loss:.2f}")
    model.eval()


Epoka: 001/003 | Partia 000/002 | Strata zbioru szkoleniowego: 0.95
Epoka: 001/003 | Partia 001/002 | Strata zbioru szkoleniowego: 0.85
Epoka: 002/003 | Partia 000/002 | Strata zbioru szkoleniowego: 0.00
Epoka: 002/003 | Partia 001/002 | Strata zbioru szkoleniowego: 0.01
Epoka: 003/003 | Partia 000/002 | Strata zbioru szkoleniowego: 0.00
Epoka: 003/003 | Partia 001/002 | Strata zbioru szkoleniowego: 0.00


In [14]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.9415, -4.5816],
        [ 2.5763, -4.0435],
        [ 2.0692, -3.2946],
        [-2.4601,  3.5966],
        [-2.9257,  4.2620]])


In [15]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[0.9995, 0.0005],
        [0.9987, 0.0013],
        [0.9953, 0.0047],
        [0.0023, 0.9977],
        [0.0008, 0.9992]])


In [17]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


### Funkcja obliczająca dokładność

In [18]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0
    for idx, (features,labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
    return (correct / total_examples).item()

In [19]:
print(compute_accuracy(model,train_loader))

1.0


In [21]:
print(compute_accuracy(model,test_loader))

0.5


### Zapisywanie modelu

In [22]:
torch.save(model.state_dict(), "model.pth")